In [1]:
# imports
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt    
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
import matplotlib.pyplot as plt

In [2]:
# Train Data
dircTrain = 'Landscape_Classification\Training_Data'
dataTrain = tf.keras.utils.image_dataset_from_directory(dircTrain)
dataTrain = dataTrain.map(lambda x,y: (x/255,y))
# dataTrain.as_numpy_iterator().next()[0].max() #to print max == 1 (should be)
# print(len(dataTrain))
train_size = int(len(dataTrain))
train = dataTrain.take(train_size)

Found 10000 files belonging to 5 classes.


In [3]:
# Validation Data
dircV = 'Landscape_Classification\Validation_Data'
dataV = tf.keras.utils.image_dataset_from_directory(dircV)
dataV = dataV.map(lambda x,y: (x/255,y))
# dataV.as_numpy_iterator().next()[0].max()
# len(dataV)
val_size = int(len(dataV))
val = dataV.take(val_size)

Found 1500 files belonging to 5 classes.


In [7]:
# Model

model = Sequential()
model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(256,256,3)))
model.add(MaxPooling2D())
model.add(Conv2D(32, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(16, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(5, activation='softmax'))

# Model Compile

model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.001),
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=[tf.metrics.SparseCategoricalAccuracy()],
)

# Model Summery if needed

# model.summary()

In [9]:
#Train Model
model.fit(train, epochs=3, validation_data=val)

Epoch 1/3
313/313 [==============================] - 244s 779ms/step - loss: 0.8724 - sparse_categorical_accuracy: 0.6657 - val_loss: 0.8777 - val_sparse_categorical_accuracy: 0.6520
Epoch 2/3
313/313 [==============================] - 251s 801ms/step - loss: 0.7468 - sparse_categorical_accuracy: 0.7193 - val_loss: 0.9123 - val_sparse_categorical_accuracy: 0.6567
Epoch 3/3
313/313 [==============================] - 256s 817ms/step - loss: 0.6262 - sparse_categorical_accuracy: 0.7670 - val_loss: 0.9066 - val_sparse_categorical_accuracy: 0.6753


In [10]:
#evaluate
model.evaluate(val)

47/47 [==============================] - 6s 126ms/step - loss: 0.9066 - sparse_categorical_accuracy: 0.6753


[0.9066324234008789, 0.6753333210945129]

In [11]:
#Save Model
model.save("Landscape_Classification")

INFO:tensorflow:Assets written to: Landscape_Classification\assets


INFO:tensorflow:Assets written to: Landscape_Classification\assets


In [14]:
#Testing
model = tf.keras.models.load_model('Landscape_Classification')
img = tf.keras.utils.load_img(
    'Landscape_Classification/Testing_Data/Coast/Coast-Test(1).jpeg',#path of testing image
    target_size=(256, 256)
)

%matplotlib inline

img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

# Make a prediction
predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])
print(score)

# Print the predictions
class_names=["Coast", "Desert", "Forest", "Glacier", "Mountain"]
print("This image most likely belongs to {} with a {:.2f} percent confidence."
      .format(class_names[np.argmax(score)], 100 * np.max(score)))

1/1 [==============================] - 0s 65ms/step
tf.Tensor([0.4046097 0.1488476 0.1488476 0.1488476 0.1488476], shape=(5,), dtype=float32)
This image most likely belongs to Coast with a 40.46 percent confidence.
